In [1]:
import spacy
import pandas as pd
import numpy as np
import neuralcoref

data_raw = pd.read_csv('pictures_sample.csv')
data_english = data_raw[data_raw.Language == 'EN']
def author_del(text):
    text_s = text
    while(text_s[-1] != '\n' and len(text_s) > 1): 
        text_s = text_s[:-1]
    if(len(text_s) <= 1):
        return(text)
    return(text_s[:-1])
data_english.Description = data_english.Description.apply(author_del)          
data_english.drop_duplicates(subset='Description', keep='first', inplace=True)
ind = list(data_english.index)

print(data_english.shape)


C:\Users\THINKPAD\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
C:\Users\THINKPAD\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)
C:\Users\THINKPAD\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (3,21,22,33,35,39,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\THINKPAD\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

(82608, 54)


In [2]:
ent_load = pd.read_pickle("ent.csv")
M = ent_load.to_numpy()
M[M == None] = 0

for i in range(M.shape[0]):
    for j in range(M.shape[1]):
        if M[i,j] == 0:
            M[i,j] = []
for i in range(M.shape[0]):
    for j in range(M.shape[1]):
        if M[i,j] != []:
            M[i,j] = [M[i,j]]
            
M = list(M.sum(axis = 1))
scoring = pd.read_csv('scoring.csv', delimiter = ";")
dict_val = {}

for i in range(48):
    dict_val[scoring['function'][i]] = scoring['score_norm'][i]
    
def dep_to_val(l):
    L = []
    for i in range(len(l)):
        if l[i] in dict_val.keys():
            L.append(dict_val[l[i]])
        else:
            L.append(0)
    return(max(L))

def final(l):
    L = [] 
    for i in l:
        ll = [] 
        for ii in i:
            ll.append((ii[0], dep_to_val(ii[1])))
        L.append(ll)
    for i in range(len(L)):
        name = L[i][0][0]
        score = 0
        for ii in L[i]:
            score += ii[1]
        L[i] = (name, score)
    return(L)

def clean_final(l):
    L = []
    for i in l:
        ll = []
        for ii in i:
            if ii not in ll:
                ll.append(ii)
        L.append(ll)
    return(L)

M = [final(m) for m in M]
M = clean_final(M)
M = [dict(m) for m in M]

In [3]:
nlp=spacy.load("en_core_web_md")
neuralcoref.add_to_pipe(nlp,greedyness=0.5)

In [4]:
def ent_in_words(i, j, doc):
    if(j+i-1 >= len(doc)):
        return("")
    words = ""
    for k in range(i):
        words += (doc[j+k].text + " ")
    return(words[:-1])

def dep_ent(ent, doc):
    start= ent.start
    end=ent.end
    for k in range(start,end):
        if doc[k].head.text not in ent.text: 
            if doc[k].dep_=='conj':   #réfléchir si vraiment optimal et si on doit le faire pour compound...
                tok=doc[k]            
                while tok.dep_=='conj':
                    tok=tok.head      #réfléchir à si on se décale latéralement de 1 mot/entité à place de faire .head
                return(tok.dep_)
            
            if doc[k].dep_=='compound':   #réfléchir si vraiment optimal et si on doit le faire pour compound...
                tok=doc[k]            
                while tok.dep_=='compound':
                    tok=tok.head      #réfléchir à si on se décale latéralement de 1 mot/entité à place de faire .head
                return(tok.dep_)
            return(doc[k].dep_)
        
    return 0

def sort_ent(doc):
    ent_coref=[ent for ent in doc.ents if ent._.is_coref and ent_good_type(ent)]
    ent_vanilla=[ent for ent in doc.ents if ent_good_type(ent) and not ent._.is_coref]
    return ent_coref + ent_vanilla

def scores_doc(doc):
    res={}
    for ent in doc.ents:
        if ent.text not in res.keys():
            res[ent.text]=dict_val[dep_ent(ent,doc)]
        else:
            res[ent.text]+=dict_val[dep_ent(ent,doc)]
    return res

def scores_doc_coref1(doc):
    doc=nlp(doc._.coref_resolved)
    res={}
    for ent in doc.ents:
        if ent.text not in res.keys():
            res[ent.text]=dict_val[dep_ent(ent,doc)]
        else:
            res[ent.text]+=dict_val[dep_ent(ent,doc)]
    return res

def is_in_cluster(ent,cluster):  #détermine si une entité est dans une des mentions d'un cluster
    for span in cluster.mentions:
        if ent.text in span.text:
            return True
    return False

def ent_good_type(ent): #filtre les entités selon leur type
    return (ent.label_ == "PERSON"or ent.label_ == "NORP" or ent.label_ == "ORG" or ent.label_ == "GPE" or ent.label_ == "EVENT" or ent.label_ == "LOC")

def scores_doc_coref2(doc):
    clusters=doc._.coref_clusters  #liste des clusters du doc. Un cluster est de la forme Trump : [Trump, he]
    res={} #le resultat sera un dictionnaire qui permet d'associer un score à chque entité
    ent_treated={} #dictionnaire pour différencier les entités traitées des autres
    ents_sorted=sort_ent(doc)  #on place les entités ayant une coref avant les autres
    for ent in ents_sorted:   
        ent_treated[ent.text]=0  #on initialise en affectant 0 à toutes les entités

    for ent in ents_sorted:     #pour chaque entité
        if ent_treated[ent.text]==0: #si elle n'est pas considérée comme "traitée"
            if ent._.is_coref:  #si elle est coréférencée (donc si elle apparait en tant que mention pour un cluster )
                #print(ent)
                for cluster in clusters:  #on va chercher le cluster associé à l'entité puisqu'elle est coref
                    if ent in cluster.mentions: # on regarde si l'entité est dans les mentions du cluster
                        for span in cluster.mentions: #Mtnt qu'on est dans le bon cluster, on regarde pour chaque span
                            if span.ents != [] : 
                                max_sim=ent.similarity(span.ents[0])
                                max_span_ent=span.ents[0]
                                for span_ent in span.ents: #on cherche l'entité dans le span la plus proche de ent
                                    if ent.similarity(span_ent) > max_sim:
                                        max_span_ent=span_ent
                                        max_sim=ent.similarity(span_ent)
                                ent_treated[max_span_ent.text]=1 #une fois qu'on l'a trouvé on la marque comme traitée
                            else:
                                max_span_ent=span        
                            if ent.text not in res.keys(): 
                                res[ent.text]=dict_val[dep_ent(max_span_ent,doc)]  #on affecte le score de l'entité
                            else:
                                res[ent.text] += dict_val[dep_ent(max_span_ent,doc)] 
            else: #si l'entité n'est pas exactement coréférencée par neuralcoref
                flag=0
                for cluster in clusters:
                    if is_in_cluster(ent,cluster) and ent.label_ != 'NORP': 
                        flag=1
                        for span in cluster.mentions:
                            #print(span)
                            if span.ents != [] :
                                max_sim=ent.similarity(span.ents[0])
                                max_span_ent=span.ents[0]
                                for span_ent in span.ents:
                                    if ent.similarity(span_ent) > max_sim:
                                        max_span_ent=span_ent
                                        max_sim=ent.similarity(span_ent)
                                ent_treated[max_span_ent.text]=1
                            else:
                                max_span_ent=span #on peut peut etre supprimer cette branche else et...
                            if ent.text not in res.keys():
                                res[ent.text]=dict_val[dep_ent(max_span_ent,doc)] #...replacer max_span_ent par span ici
                            else:
                                res[ent.text] += dict_val[dep_ent(max_span_ent,doc)]  #et ici
                if flag==0: #si l'entité n'est vraiment dans aucun cluster   
                    if ent.text not in res.keys():
                        res[ent.text]=dict_val[dep_ent(ent,doc)]
                    else:
                        res[ent.text]+=dict_val[dep_ent(ent,doc)] 
    #print(ent_treated)
    return res
    
doc=nlp('Today, Donald Trump met Macron in France. Trump wants this country to be happy.')
#doc=nlp(data_english.Description[4])
dep_ent(doc.ents[0],doc)
print(doc)
print(doc.ents)
print(scores_doc_coref2(doc))
print(scores_doc_coref1(doc))
print(scores_doc(doc))


Today, Donald Trump met Macron in France. Trump wants this country to be happy.
(Today, Donald Trump, Macron, France, Trump)
{'Donald Trump': 2.0, 'France': 1.829787234, 'Macron': 0.872340426}
{'Today': 0.70212766, 'Donald Trump': 2.0, 'Macron': 0.872340426, 'France': 1.829787234}
{'Today': 0.70212766, 'Donald Trump': 1.0, 'Macron': 0.872340426, 'France': 0.8297872340000001, 'Trump': 1.0}


In [5]:


#doc=nlp("The owners of a Tucson-area Mexican restaurant are fending off social media attacks after appearing in the VIP area at a Trump rally in Phoenix last month. The Arizona Daily Star reported that a Facebook group posted a screenshot image from the rally that showed Sammy’s Mexican Grill co-owner Betty Rivas standing behind Trump, wearing a red cowboy hat emblazoned with Latinos Love Trump. The newspaper says the post attracted more than 230 comments, almost all negative. Sammy’s co-owner Jorge Rivas said some people also posted “very ugly stuff” on social media including the restaurant’s Yelp and Google pages. On Sunday Trump tweeted support for the Rivas’ restaurant. Despite apparently watching a segment on the eatery on his favourite Fox News magazine show, Fox & Friends, the president got its location wrong. The food is GREAT at Sammy’s Mexican Grill in Phoenix, Arizona, the tweet said. Congratulations to Betty & Jorge Rivas on doing such a wonderful job. I will try hard to stop by the next time I am in Phoenix. Phoenix and Tucson are 113 miles apart. Trump’s hardline immigration policies make him unpopular with most Latino Americans. Rivas said he and his wife posted a video on Facebook defending their rights as naturalized American citizens to vote, support and meet whomever they please. Just because we are Latinos it doesn’t mean that we have to feel like every other Latino in this country, Rivas said. We are individuals and we feel that we have the constitutional right to meet and support whoever we want. Jorge Rivas said the online attacks had little to no impact on business at the restaurant which he and his wife opened in 1996.")

doc=nlp(data_english.Description[0])
print(doc)
print('\n')
print(doc.ents)
print('\n')
for X in doc.ents:
    print(X,dict_val[dep_ent(X,doc)],dep_ent(X,doc))
print('\n')

print(doc._.coref_clusters)
print('\n')

L=scores_doc_coref2(doc)
print("Résultat méthode 2 : ", dict(sorted(L.items(), key=lambda t: t[1])))
print('\n')

L=scores_doc_coref1(doc)
print("Résultat méthode 1 : ", dict(sorted(L.items(), key=lambda t: t[1])))
print('\n')

L=scores_doc(doc)
print("Résultat sans neuralcoref : ", dict(sorted(L.items(), key=lambda t: t[1])))


US-British actress Sienna Miller poses after unveiling her dedicated beach locker room on the Promenade des Planches at the 45th Deauville US Film Festival in Deauville, western France, on September 11, 2019. 


(US, British, Sienna Miller, the Promenade des Planches, 45th, Deauville US Film Festival, Deauville, France, September 11, 2019)


US 0.489361702 amod
British 0.489361702 amod
Sienna Miller 1.0 nsubj
the Promenade des Planches 0.8297872340000001 pobj
45th 0.489361702 amod
Deauville US Film Festival 0.8297872340000001 pobj
Deauville 0.8297872340000001 pobj
France 0.46808510600000003 appos
September 11, 2019 0.8297872340000001 pobj


[US-British actress Sienna Miller: [US-British actress Sienna Miller, her]]


Résultat méthode 2 :  {'France': 0.46808510600000003, 'British': 0.489361702, 'Deauville US Film Festival': 0.8297872340000001, 'Deauville': 0.8297872340000001, 'US': 0.872340425, 'Sienna Miller': 1.382978723}


Résultat méthode 1 :  {'France': 0.46808510600000003, '45th':

In [6]:
def score_sim(doc,image):
    res=0
    score_doc=scores_doc_coref2(doc)
    score_image=scores_doc_coref2(image)
    for i in score_doc.keys():
        if i in score_image:
            res += (score_doc[i]+score_image[i])/2
    return res

def best_image(doc):
    best_score=0
    best_descr=''
    for descr in data_english.Description:
        if score_sim(doc,descr) > best_score:
            best_score=score_sim
            best_descr=descr
    return best_descr

In [11]:
import time

start_time=time.time()

nlp_list = []
for desc in data_english.Description[:10000]:
    nlp_list.append(nlp(desc))

print("Temps d'exécution : ", time.time()-start_time, "s")

Temps d'exécution :  832.3939981460571 s


In [18]:
start_time=time.time()
for desc in data_english.Description[10000:20000]:
    nlp_list.append(nlp(desc))
print("Temps d'exécution : ", time.time()-start_time, "s")

Temps d'exécution :  944.046897649765 s


In [23]:
start_time=time.time()
for desc in data_english.Description[20000:30000]:
    nlp_list.append(nlp(desc))
print("Temps d'exécution : ", time.time()-start_time, "s")

Temps d'exécution :  848.4053909778595 s


In [24]:
start_time=time.time()
for desc in data_english.Description[30000:40000]:
    nlp_list.append(nlp(desc))
print("Temps d'exécution : ", time.time()-start_time, "s")

Temps d'exécution :  858.3026707172394 s


In [25]:
start_time=time.time()
for desc in data_english.Description[40000:50000]:
    nlp_list.append(nlp(desc))
print("Temps d'exécution : ", time.time()-start_time, "s")

Temps d'exécution :  836.3889255523682 s


In [28]:
start_time=time.time()
for desc in data_english.Description[50000:65000]:
    nlp_list.append(nlp(desc))
print("Temps d'exécution : ", time.time()-start_time, "s")


Temps d'exécution :  1327.91654586792 s


In [30]:
start_time=time.time()
for desc in data_english.Description[65000:75000]:
    nlp_list.append(nlp(desc))
print("Temps d'exécution : ", time.time()-start_time, "s")

Temps d'exécution :  760.1543135643005 s


In [31]:
start_time=time.time()
for desc in data_english.Description[75000:]:
    nlp_list.append(nlp(desc))
print("Temps d'exécution : ", time.time()-start_time, "s")

Temps d'exécution :  676.7457978725433 s


In [32]:
score_list = [scores_doc_coref2(doc) for doc in nlp_list]

#final_list=[]
#for k in range(len(nlp_list)):
#    descr=nlp_list[k]
#    liste_score=[]
#    score_doc=scores_doc_coref2(descr)
#    for ent in score_doc.keys():
#        liste_score.append([ent,score_doc[ent]])
#     
#    final_list.append(list_score[k])

#ent_df = pd.DataFrame(score_list, dtype = 'O')

# Pour sauvegarder ce DataFrame il faut utiliser la méthode to_pickle et non pas to_csv

#ent_df.to_pickle("ent.csv")


KeyError: 0

In [ ]:
ent_load = pd.read_pickle("ent.csv")

In [ ]:
score_doc(doc,doc)

In [ ]:
spacy.__version__

        